In [1]:
import pandas as pd
import numpy as np

In [2]:
# df = pd.read_csv('/home/daniil/Рабочий стол/CLASSIFICATION PROBLEM/JobMatchRecommenderSystem/Scripts/Ready_to_use_data').fillna(' ')
# df = df.iloc[np.where((df.department != ' ') & (df.unit != ' '))]

In [2]:
df = pd.read_excel('/home/daniil/Рабочий стол/CLASSIFICATION PROBLEM/JobMatchRecommenderSystem/Scripts/train_data_05-03.xlsx').fillna(' ')
df = df.iloc[np.where((df.client_department != ' ') & (df.client_unit != ' '))]

In [9]:
df.catalogue_code.nunique()

1545

In [91]:
# Создаём словарь с нумерацией функций 1 уровня
func_1_classes = {}
for num, function in enumerate( df.func_1.unique() ):
    func_1_classes[function] = num

In [99]:
# Отбираем описание и класс(func_1) для дальнейшей обработки
data = df[['full_description','func_1']].to_numpy()

#### TF-IDF(напоминание)

https://s1.hostingkartinok.com/uploads/images/2024/01/17e222c8f02e0e6bd647c87cc8c93eba.jpg

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### Некоторые параметры: 
* input : string {‘filename’, ‘file’, ‘content’}
*  lowercase : boolean, default True
*  preprocessor : callable or None (default)
*  tokenizer : callable or None (default)
*  stop_words : string {‘english’}, list, or None (default)
*  ngram_range : tuple (min_n, max_n)
*  max_df : float in range [0.0, 1.0] or int, default=1.0
*  min_df : float in range [0.0, 1.0] or int, default=1
*  max_features : int or None, default=None

In [101]:
# Примняем TF-IDF. Получаемая матрица по строкам соответствует кол-ву наблюдений 
# и размерности словаря по столбцам.
vectorizer = TfidfVectorizer() 
vectors = vectorizer.fit_transform(data[:,0])


# Вместо текста вставляем векторизацию посредством TF IDF
vectors = np.array(vectors.todense())
vectors.shape

(11030, 1479)

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression, SGDClassifier

# В качестве признаков выступает векторизованное описание, в качестве лейблов выступают функции 1 уровня.
X_train, X_test, y_train, y_test = train_test_split(vectors, data[:,1], test_size=0.2,\
                                                    random_state=0, shuffle=True)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8824, 1479), (2206, 1479), (8824,), (2206,))

##### LogReg

In [103]:
model = LogisticRegression(penalty='l2', n_jobs=-1, solver='lbfgs')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность Логистической регрессии:{acc}')

Точность Логистической регрессии:0.756572982774252


/home/daniil/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### SGDClassifier

In [104]:
model = SGDClassifier(loss='hinge', alpha=0.0001, max_iter=500, n_jobs=-1)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность Логистической регрессии:{acc}')

Точность Логистической регрессии:0.7778785131459656


##### LinearSVC

In [105]:
model = svm.LinearSVC(loss='squared_hinge', multi_class='ovr')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность Логистической регрессии:{acc}')

Точность Логистической регрессии:0.7973708068902992


##### SVC

In [106]:
model = svm.SVC( kernel='rbf')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность Логистической регрессии:{acc}')

Точность Логистической регрессии:0.8209428830462375


# Вторая чать эксперимента: разобъём объекты по группам в зависимости от функций 1 уровня и для каждого из них натрпнируем классификатор.

In [4]:
df.head()

,client_department,client_unit,client_job_name,full_description,catalogue_code,func_1,func_2,func_3
0,маркетинг дизайн и онлайн продажи,департамент онлайн продаж,руководитель департамента,маркетинг дизайн и онлайн продаж департамент р...,MAR_E_20_3,Маркетинг,Электронная коммерция,Электронная коммерция
2,управление крупным частным капиталом,департамент по управлению крупным частным капи...,руководитель департамента,управлен крупн частн капитал департамент по ру...,IB_M_1_3,Инвестиционная деятельность,Общее управление функцией,Общее управление функцией
3,крупный корпоративный и инвестиционный бизнес,департамент по работе с крупными корпоративным...,старший вице президент руководитель департамента,крупн корпоративн и инвестицион бизнес департа...,CB_M_1_3,Корпоративный бизнес,Общее управление функцией,Общее управление функцией
4,управление рисками,департамент рисков розничного и малого бизнеса,руководитель департамента,управлен риск департамент розничн и мал бизнес...,R_M_1_3,Управление рисками,Общее управление функцией,Общее управление функцией
5,малый и микро корпоративный бизнес,департамент продаж и развития малого и микроби...,старший вице президент руководитель департамента,мал и микр корпоративн бизнес департамент прод...,CB_M_1_3,Корпоративный бизнес,Общее управление функцией,Общее управление функцией


In [29]:
for func in df.func_1.unique():
    data = df[df.func_1 == func]
    print(f'Наблюдений для данной функции 1 уровня: {data.shape[0]}')
    print(f'Кол-во уникальных кодов для данной группы: {data.func_2.nunique()}')
    
    data = data[['full_description','func_2']].to_numpy()
    if np.all(data[:,1] == data[:,1][0]):
        continue 
    vectorizer = TfidfVectorizer() 
    vectors = vectorizer.fit_transform(data[:,0])
    

    # Вместо текста вставляем векторизацию посредством TF IDF
    vectors = np.array(vectors.todense())
    X_train, X_test, y_train, y_test = train_test_split(vectors, data[:,1], test_size=0.2,\
                                                    random_state=0, shuffle=True)
    model = svm.SVC( kernel='rbf')
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    acc = accuracy_score(predictions, y_test)

    print(f'Точность Логистической регрессии:{acc}')
    print('--------------------------------------')

Наблюдений для данной функции 1 уровня: 564
Кол-во уникальных кодов для данной группы: 10
Точность Логистической регрессии:0.7787610619469026
--------------------------------------
Наблюдений для данной функции 1 уровня: 29
Кол-во уникальных кодов для данной группы: 2
Точность Логистической регрессии:1.0
--------------------------------------
Наблюдений для данной функции 1 уровня: 1196
Кол-во уникальных кодов для данной группы: 7
Точность Логистической регрессии:0.9208333333333333
--------------------------------------
Наблюдений для данной функции 1 уровня: 252
Кол-во уникальных кодов для данной группы: 5
Точность Логистической регрессии:0.8627450980392157
--------------------------------------
Наблюдений для данной функции 1 уровня: 1352
Кол-во уникальных кодов для данной группы: 7
Точность Логистической регрессии:0.8265682656826568
--------------------------------------
Наблюдений для данной функции 1 уровня: 134
Кол-во уникальных кодов для данной группы: 3
Точность Логистической р

ValueError: The number of classes has to be greater than one; got 1 class

In [28]:
np.all(data[:,1] == data[:,1][0])

True

In [15]:
# Отбираем описание и класс(func_1) для дальнейшей обработки
semi_data = df[df.func_1 == 'Инвестиционная деятельность']
semi_data = semi_data[['full_description','func_2']].to_numpy()

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Примняем TF-IDF. Получаемая матрица по строкам соответствует кол-ву наблюдений 
# и размерности словаря по столбцам.
vectorizer = TfidfVectorizer() 
vectors = vectorizer.fit_transform(semi_data[:,0])


# Вместо текста вставляем векторизацию посредством TF IDF
vectors = np.array(vectors.todense())
vectors.shape

(29, 69)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression, SGDClassifier

# В качестве признаков выступает векторизованное описание, в качестве лейблов выступают функции 1 уровня.
X_train, X_test, y_train, y_test = train_test_split(vectors, semi_data[:,1], test_size=0.2,\
                                                    random_state=0, shuffle=True)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23, 69), (6, 69), (23,), (6,))

In [18]:
model = LogisticRegression(penalty='l2', n_jobs=-1, solver='lbfgs')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность Логистической регрессии:{acc}')

Точность Логистической регрессии:1.0


In [19]:
model = SGDClassifier(loss='hinge', alpha=0.0001, max_iter=500, n_jobs=-1)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность SGDClassifier:{acc}')

Точность SGDClassifier:1.0


In [20]:
model = svm.SVC( kernel='rbf')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность Логистической регрессии:{acc}')

Точность Логистической регрессии:1.0
